In [1]:
import numpy as np

from jitr import reactions, rmatrix
from jitr.utils.kinematics import classical_kinematics

In [2]:
def potential_scalar(r, depth, mass):
    return -depth * np.exp(-r / mass)

In [3]:
def coupling_2level(l):
    r"""
    Each partial wave has 2 uncoupled channels
    """
    return np.array([[1, 0], [0, 1]])

In [4]:
def potential_2level(r, depth, mass, coupling):
    r"""
    if coupling=0, this 2 level interaction acts on each
    channel independently
    """
    diag = potential_scalar(r, depth, mass)
    off_diag = potential_scalar(r, coupling, mass)
    return np.array(
        [[diag, off_diag], [off_diag, diag]],
    )

In [5]:
nbasis = 40
solver = rmatrix.Solver(nbasis)

In [6]:
nchannels = 2
sys_2level = reactions.ProjectileTargetSystem(
    channel_radius=5 * np.pi,
    lmax=10,
    mass_target=44657,
    mass_projectile=938.3,
    Ztarget=20,
    Zproj=1,
    coupling=coupling_2level,
)

In [7]:
l = 0
Elab = 42.1
Ecm, mu, k, eta = classical_kinematics(
    sys_2level.mass_target,
    sys_2level.mass_projectile,
    Elab,
    sys_2level.Zproj * sys_2level.Ztarget,
)
channels, asymptotics = sys_2level.get_partial_wave_channels(Ecm, mu, k, eta)

get coupled channels for partial wave l

In [8]:
channels_coupled = channels[l]
asymptotics_coupled = asymptotics[l]

get un-coupled channels for partial wave l

In [9]:
channels_uncoupled = channels[l].decouple()
asymptotics_uncoupled = asymptotics[l].decouple()

un-coupled scalar subsystems

In [10]:
params_2level = (10, 4, 0)
params_scalar = (10, 4)

In [11]:
b = solver.precompute_boundaries(sys_2level.channel_radius)
free = solver.free_matrix(channels_coupled.a, channels_coupled.l, channels_coupled.E)
interaction = solver.interaction_matrix(
    channels_coupled.k[0],
    channels_coupled.E[0],
    channels_coupled.a,
    channels_coupled.size,
    potential_2level,
    params_2level,
)

# test diaginal blocks
free_0 = solver.free_matrix(
    channels_uncoupled[0].a,
    channels_uncoupled[0].l,
    channels_uncoupled[0].E,
)
free_1 = solver.free_matrix(
    channels_uncoupled[1].a,
    channels_uncoupled[1].l,
    channels_uncoupled[1].E,
)

np.testing.assert_almost_equal(
    free_0,
    solver.get_channel_block(free, 0, 0),
)
np.testing.assert_almost_equal(
    free_1,
    solver.get_channel_block(free, 1, 1),
)

np.testing.assert_almost_equal(
    solver.interaction_matrix(
        channels_uncoupled[0].k[0],
        channels_uncoupled[0].E[0],
        channels_uncoupled[0].a,
        channels_uncoupled[0].size,
        potential_scalar,
        params_scalar,
    ),
    solver.get_channel_block(
        interaction,
        0,
        0,
    ),
)
np.testing.assert_almost_equal(
    solver.interaction_matrix(
        channels_uncoupled[1].k[0],
        channels_uncoupled[1].E[0],
        channels_uncoupled[1].a,
        channels_uncoupled[1].size,
        potential_scalar,
        params_scalar,
    ),
    solver.get_channel_block(
        interaction,
        1,
        1,
    ),
)

# test off diag blocks
for i in range(nchannels):
    for j in range(nchannels):
        if j != i:
            np.testing.assert_almost_equal(solver.get_channel_block(free, i, j), 0)
            np.testing.assert_almost_equal(
                solver.get_channel_block(interaction, i, j), 0
            )

# test full matrix
A = (
    solver.interaction_matrix(
        channels_uncoupled[0].k[0],
        channels_uncoupled[0].E[0],
        channels_uncoupled[0].a,
        channels_uncoupled[0].size,
        potential_scalar,
        params_scalar,
    )
    + free_0
)
Am = free + interaction
np.testing.assert_almost_equal(Am[:nbasis, :nbasis], A)
bm = np.hstack([b, b])
x = np.linalg.solve(A, b)
xm = np.linalg.solve(Am, bm)
np.testing.assert_almost_equal(x, xm[:nbasis])

In [12]:
R, S, u = solver.solve(
    channels_uncoupled[0],
    asymptotics_uncoupled[0],
    potential_scalar,
    params_scalar,
    weights=np.array([1, 1]),
)
R2, S2, u2 = solver.solve(
    channels_uncoupled[1],
    asymptotics_uncoupled[1],
    potential_scalar,
    params_scalar,
)

# solve the full system
Rm, Sm, xm = solver.solve(
    channels_coupled,
    asymptotics_coupled,
    potential_2level,
    params_2level,
)
np.testing.assert_almost_equal(np.linalg.det(Sm.conj().T @ Sm), 1)
np.testing.assert_almost_equal(np.linalg.det(S.conj().T @ S), 1)
np.testing.assert_almost_equal(Sm[1, 0], 0)
np.testing.assert_almost_equal(Sm[0, 1], 0)
np.testing.assert_almost_equal(Rm[1, 0], 0)
np.testing.assert_almost_equal(Rm[0, 1], 0)
np.testing.assert_almost_equal(Sm[1, 1], S2)
np.testing.assert_almost_equal(Rm[1, 1], R2)
np.testing.assert_almost_equal(Sm[0, 0], S)
np.testing.assert_almost_equal(Rm[0, 0], R)

In [13]:
a = sys_2level.channel_radius
print(a)


def Z(asymptotics, R):
    return (
        np.diag(asymptotics.Hp) - R * asymptotics.Hpp[:, np.newaxis] * a,
        np.diag(asymptotics.Hm) - R * asymptotics.Hmp[:, np.newaxis] * a,
    )

15.707963267948966


In [14]:
Z(asymptotics_coupled, Rm)

(array([[0.98845593+0.72791984j, 0.        +0.j        ],
        [0.        +0.j        , 0.98845593+0.72791984j]]),
 array([[0.98845593-0.72791984j, 0.        +0.j        ],
        [0.        +0.j        , 0.98845593-0.72791984j]]))

In [15]:
Z(asymptotics_uncoupled[0], R)

(array([[0.98845593+0.72791984j]]), array([[0.98845593-0.72791984j]]))

In [16]:
Z(asymptotics_uncoupled[1], R2)

(array([[0.98845593+0.72791984j]]), array([[0.98845593-0.72791984j]]))

In [17]:
Rm

array([[0.04448574+0.j, 0.        +0.j],
       [0.        +0.j, 0.04448574+0.j]])

In [18]:
R

array([[0.04448574+0.j]])

In [19]:
R2

array([[0.04448574+0.j]])

In [20]:
asymptotics_coupled.Hpp

array([-0.92771393+0.32849075j, -0.92771393+0.32849075j])

In [21]:
asymptotics_uncoupled[0].Hp

array([0.34018764+0.95746273j])

In [22]:
R * (asymptotics_uncoupled[0].Hpp)[:, np.newaxis]

array([[-0.04127004+0.01461315j]])

In [23]:
(
    np.diag(asymptotics_uncoupled[0].Hp)
    - R * (asymptotics_uncoupled[0].Hpp)[:, np.newaxis] * a
)

array([[0.98845593+0.72791984j]])

In [24]:
np.diag(asymptotics_uncoupled[0].Hp)

array([[0.34018764+0.95746273j]])

In [25]:
np.diag(asymptotics_coupled.Hp)

array([[0.34018764+0.95746273j, 0.        +0.j        ],
       [0.        +0.j        , 0.34018764+0.95746273j]])